In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aichallenge2020/training.csv
/kaggle/input/aichallenge2020/sampleSubmission.csv
/kaggle/input/aichallenge2020/test.csv


In [2]:
#Loading training and test sets

train_set = pd.read_csv("/kaggle/input/aichallenge2020/training.csv")
test_set = pd.read_csv("/kaggle/input/aichallenge2020/test.csv")

pd.set_option('display.max_rows', None)

In [3]:
#display(train_set)

Preprocessing

In [4]:
# Droping duplicated tweets

train_set = train_set.drop_duplicates(subset='tweet', keep='first')

In [5]:
# Checking for duplicated tweets
#train_set.tweet.duplicated().sum()

In [6]:
# Loading pre-trained word embeddings
import spacy
!python -m spacy download es_core_news_lg

     |████████████████████████████████| 573.1 MB 66.2 MB/s 
  Created wheel for es-core-news-lg: filename=es_core_news_lg-2.3.1-py3-none-any.whl size=573139080 sha256=b25588e3105e7f35014166913b354ec6758ae80eaffb009799e7e0de0b17cb20
  Stored in directory: /tmp/pip-ephem-wheel-cache-lralfrd_/wheels/a5/c7/e8/a49f20406fb1f1f557639d51ce800bdd12241f91e1f21f1e91
Successfully built es-core-news-lg
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_lg')


In [7]:
import es_core_news_lg

#____-----____-----____-----____-----____-----____-----____-----____-----

def get_embeddings(vocab):
    return vocab.vectors.data

nlp = es_core_news_lg.load()
embeddings = get_embeddings(nlp.vocab)

#print(embeddings.shape)

# Cleans the tweet
def tweet_cleaner(tok_tweet):
    tweet = " "
    tweet = [tweet.join(str(tok.text) for tok in tok_tweet if tok.is_stop==False and tok.like_url==False)]
    return tweet
#____-----____-----____-----____-----____-----____-----____-----____-----

# Preprocesses every tweet with spacy model
def preprocess_pipe(set):
    tweets = []
    for tok_tweet in nlp.pipe(set['tweet'], batch_size=20):
        tweets.append(tweet_cleaner(tok_tweet))
    tweets = np.array([str(tweet) for tweet in tweets])
    return pd.DataFrame({'tweet': tweets})

#____-----____-----____-----____-----____-----____-----____-----____-----

# Reprocesses clean tweets and vectorizes tweets in order to feed them to the model
def tok_vec(set):
    #tweets = preprocess_pipe(set)
    tok_set = nlp.pipe(set['tweet'], batch_size = 20)
    vec_set = get_features(tok_set,25)
    return vec_set

#____-----____-----____-----____-----____-----____-----____-----____-----

def get_features(docs, max_length):
    docs = list(docs)
    Xs = np.zeros((len(docs), max_length), dtype="int32")
    for i, doc in enumerate(docs):
        j = 0
        for token in doc:
            vector_id = token.vocab.vectors.find(key=token.orth)
            if vector_id >= 0:
                Xs[i, j] = vector_id
            else:
                Xs[i, j] = 0
            j += 1
            if j >= max_length:
                break
    return Xs

In [8]:
# Preprocessing train and test sets

vec_train_set = tok_vec(train_set)
#print(vec_train_set.shape)
vec_test_set = tok_vec(test_set)
#print(vec_test_set.shape)

In [9]:
# ESTADISTICAS 
import emoji

def cont_signos(tweets):
    array=[]
    dic={}
    for tweet in tweets:
        i=0
        
        for aux2 in tweet:
              
            if '\'' in aux2:
                i+=1
            if '\"' in aux2:
                i+=1
            elif '(' in aux2:
                i+=1
            elif ')' in aux2:
                i+=1
            elif '[' in aux2:
                i+=1
            elif ']' in aux2:
                i+=1
            elif ':' in aux2:
                i+=1
            elif '.' in aux2:
                i+=1
            elif ',' in aux2:
                i+=1
            elif '¿' in aux2:
                i+=1
            elif '?' in aux2:
                i+=1
            elif '¡' in aux2:
                i+=1
            elif '!' in aux2:
                i+=1
            elif ';' in aux2:
                i+=1
            elif '{' in aux2:
                i+=1
            elif '}' in aux2:
                i+=1
            elif '...' in aux2:
                i+=1
            elif '-' in aux2:
                i+=1
        array.append(i)
    dic['signos']=array 
    return(dic)

cantsignos_train=pd.DataFrame(data=cont_signos(train_set['tweet']))
cantsignos_test=pd.DataFrame(data=cont_signos(test_set['tweet']))

def veri_comillas(tweets):
    array=[]
    dic={}

    for tweet in tweets:
        i=0
        if '\"' in tweet:
            i=1
        array.append(i)
    dic["veri_comillas"]=array
    return(dic)

comillas_train=pd.DataFrame(data=veri_comillas(train_set['tweet']))
comillas_test=pd.DataFrame(data=veri_comillas(test_set['tweet']))

def contar_arrobas(tweets):
    lis=[]
    arrobas_dic={}
    for tweet in tweets:
        
        aux=tweet.split()
        
        i=0
        
        for extraer in aux:
          
            
            if '@' in extraer:
                
                i+=1
                
        lis.append(i)
    arrobas_dic["cant_arrobas"]=lis       
    return(arrobas_dic)     
               
        
count_arrobas=pd.DataFrame(data=contar_arrobas(train_set["tweet"]))
count_arrobas_test=pd.DataFrame(data=contar_arrobas(test_set["tweet"]))

def contar_hashtag(tweets):
    hash_dic={}
    lis=[]
    for tweet in tweets:
        
        aux=tweet.split()
        
        i=0
        
        
        for extraer in aux:
            if '#' in extraer:
                i+=1
               
        lis.append(i)
    
    hash_dic["cant_hash"]=lis
    return(hash_dic)  
               
count_hashtag=pd.DataFrame(data=contar_hashtag(train_set["tweet"]))
count_hashtag_test=pd.DataFrame(data=contar_hashtag(test_set["tweet"]))

def contar_emoji(tweets):
    emoji_dic={}
    auxi=[]
    for tweet in tweets:
        i=0
      
        aux=tweet.split()
       
        for extraer in aux:
            for e in extraer:
              
                if e in emoji.UNICODE_EMOJI:
                    
                    i+=1
            
    
        auxi.append(i)
               
    emoji_dic['cant_emoji']=auxi
    return (emoji_dic)
count_emoji=pd.DataFrame(data=contar_emoji(train_set['tweet']))
c_emoji_test=pd.DataFrame(data=contar_emoji(test_set['tweet']))

def count_up_case(tweets):
    up_cases={}
    up_cases_list=[]
    for tweet in tweets:
        i=0
        split_tweet=tweet.split()
        for word in split_tweet:
            if word.isupper():
                i+=1
        up_cases_list.append(i)
    up_cases['num_up_cases']=up_cases_list
    return (up_cases)
num_up_cases_train=pd.DataFrame(data=count_up_case(train_set['tweet']))
num_up_cases_test=pd.DataFrame(data=count_up_case(test_set['tweet']))

def non_vocab_token(tweets):
    n_vocab_t={}
    n_vocab_list=[]
    for tweet in tweets:
        i=0
        split_tweet=tweet.split()
        for token in split_tweet:
            token = nlp.vocab.strings[token]
            if token not in nlp.vocab:
                i+=1
        n_vocab_list.append(i)
    n_vocab_t['num_n_vocab_t']=n_vocab_list
    return n_vocab_t
num_n_vocab_t_train=pd.DataFrame(data=non_vocab_token(train_set['tweet']))
num_n_vocab_t_test=pd.DataFrame(data=non_vocab_token(test_set['tweet']))


stadis=pd.concat([count_hashtag,num_up_cases_train,num_n_vocab_t_train,cantsignos_train,count_arrobas],axis=1)
stadis_test=pd.concat([count_hashtag_test,num_up_cases_test,num_n_vocab_t_test,cantsignos_test,count_arrobas_test],axis=1)
#print(stadis)
stadis=np.array(stadis)
stadis_test=np.array(stadis_test)
stadis_norm = np.linalg.norm(stadis)
stadis = stadis/stadis_norm
stadis_test_norm = np.linalg.norm(stadis_test)
stadis_test= stadis_test/stadis_test_norm

In [10]:
# Splitting train and val set

from sklearn.model_selection import train_test_split

 
X_train2, X_val2,y_train2,y_val2= train_test_split(stadis, train_set['is_organico'], test_size=0.2,stratify=train_set['is_organico']) 

X_train, X_val, y_train, y_val= train_test_split(vec_train_set,train_set['is_organico'], test_size=0.2,stratify=train_set['is_organico'])

#print(X_train.shape)
#print(X_val.shape)
#print(y_train.shape)
#print(y_val.shape)

#print(" ")

#print(X_train2.shape)
#print(X_val2.shape)
#print(y_train2.shape)
#print(y_val2.shape)


In [11]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [12]:
# Architecture

import keras
from keras.models import Model,Sequential
from keras.layers import Dense, Input, LSTM, Bidirectional, Flatten,ReLU, Concatenate, Conv1D,Dropout,LeakyReLU
from keras.layers import BatchNormalization,Embedding,Concatenate,SpatialDropout1D,GaussianDropout,LayerNormalization
from keras.layers import GlobalAveragePooling1D,GlobalMaxPooling1D

model1 = Input(shape=(25,))
model2 = Input(shape=(5,))

x=Embedding(embeddings.shape[0],
                    embeddings.shape[1],
                    input_length=25,
                    trainable=False,
                    weights=[embeddings],
                    mask_zero=True)(model1)


x=Conv1D(64,1)(x)
x=Bidirectional(LSTM(128, return_sequences = True, go_backwards = True))(x)
x=SpatialDropout1D(0.2)(x)
x=Bidirectional(LSTM(64, return_sequences = True, go_backwards = True))(x)
x=BatchNormalization()(x)
x=Dense(256, activation='relu')(x)

#print(x)
att = Attention(25)(x)
avg_pool1 =GlobalAveragePooling1D()(x)
max_pool1 =GlobalMaxPooling1D()(x)
x=Concatenate()([att,avg_pool1, max_pool1])

y=Dense(128, activation='relu')(model2)
y=Dropout(0.2)(y)
y=Dense(100, activation='relu')(y)
y=Dropout(0.2)(y)
model_concat = Concatenate()([x,y])


model_concat=Dense(64, activation='relu')(model_concat)
model_concat=Dense(1, activation='sigmoid')(model_concat)


model=Model(inputs=[model1,model2], outputs=model_concat)

In [13]:
# Training the model

opt = keras.optimizers.Adam(lr=1e-3)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["acc"])

model.fit([X_train,X_train2],y_train, batch_size=120, epochs=30, validation_data=([X_val,X_val2], y_val))

Epoch 1/30
171/171 [==============================] - 28s 164ms/step - loss: 0.4836 - acc: 0.7633 - val_loss: 0.4860 - val_acc: 0.8090
Epoch 2/30
171/171 [==============================] - 27s 156ms/step - loss: 0.3675 - acc: 0.8280 - val_loss: 0.3799 - val_acc: 0.8337
Epoch 3/30
171/171 [==============================] - 27s 156ms/step - loss: 0.2973 - acc: 0.8664 - val_loss: 0.3351 - val_acc: 0.8490
Epoch 4/30
171/171 [==============================] - 27s 159ms/step - loss: 0.2484 - acc: 0.8927 - val_loss: 0.3079 - val_acc: 0.8611
Epoch 5/30
171/171 [==============================] - 27s 157ms/step - loss: 0.2039 - acc: 0.9131 - val_loss: 0.3139 - val_acc: 0.8685
Epoch 6/30
171/171 [==============================] - 28s 162ms/step - loss: 0.1677 - acc: 0.9296 - val_loss: 0.3096 - val_acc: 0.8726
Epoch 7/30
171/171 [==============================] - 27s 159ms/step - loss: 0.1391 - acc: 0.9432 - val_loss: 0.3407 - val_acc: 0.8756
Epoch 8/30
171/171 [==============================] - 2

In [14]:
# Making predictions

predicted_prob = model.predict([vec_test_set,stadis_test]) #Computes probability of each tweet being organic
predictions = np.round(predicted_prob).astype(int).flatten() # Rounds the probability to 0/1
submission = pd.DataFrame({'Id': test_set['Id'],'Predicted': predictions})
submission.to_csv('submission.csv', index=False) #Turns dataframe into csv file
#display(submission.head())